In [120]:
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

In [121]:
tmp = pd.read_json("swath/granules.json").reset_index()
df = pd.json_normalize(tmp['feed'][3])
df=df[["producer_granule_id", "time_start","time_end","day_night_flag","polygons"]]

In [122]:
def create_polygon_from_string(polygon_string):
    """Creates a shapely Polygon from a string of coordinates."""

    # Remove outer brackets and quotes
    coords_str = polygon_string.strip("[]'").split()

    # Convert coordinates to floats and create pairs
    coords = [[float(coords_str[i+1]), float(coords_str[i])] for i in range(0, len(coords_str), 2)] # change order

    # Ensure the polygon is closed by adding the first point to the end
    if coords and coords[0] != coords[-1]:
        coords.append(coords[0])

    # Create the Polygon object
    polygon = Polygon(coords)
    return polygon


In [123]:
df["geometry"] = df["polygons"].apply(lambda coords: create_polygon_from_string(str(coords)))
df.drop(columns=['polygons'], inplace=True)
df["time_start"] = df["time_start"].apply(lambda x: x.split('.')[0] + 'Z')
df["time_end"] = df["time_end"].apply(lambda x: x.split('.')[0] + 'Z')

In [124]:
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")
# Define AOI as a shapely Polygon
aoi_coords = [[-102.8148701375, 6.1943456775], [-13.3448605043, 6.1943456775], 
              [-13.3448605043, 49.6429910636], [-102.8148701375, 49.6429910636], 
              [-102.8148701375, 6.1943456775]]
aoi_polygon = Polygon(aoi_coords)
aoi_gdf = gpd.GeoDataFrame({'geometry': [aoi_polygon]}, crs="EPSG:4326")
filtered_gdf = gdf[gdf.intersects(aoi_polygon)]
print(filtered_gdf.shape, gdf.shape)
filtered_gdf.to_file("swath/modis_swath_cyclone_beryl.geojson", driver='GeoJSON') # find a suitable name

(549, 5) (608, 5)


In [125]:
filtered_gdf

,producer_granule_id,time_start,time_end,day_night_flag,geometry
0,MYD021KM.A2024178.0225.061.2024178151331.hdf,2024-06-26T02:25:00Z,2024-06-26T02:30:00Z,NIGHT,"POLYGON ((-15.55328 -2.52339, 4.99931 -5.53637..."
1,MYD021KM.A2024178.0350.061.2024178213745.hdf,2024-06-26T03:50:00Z,2024-06-26T03:55:00Z,BOTH,"POLYGON ((-32.3104 46.49111, -3.09097 42.47777..."
2,MYD021KM.A2024178.0355.061.2024178213723.hdf,2024-06-26T03:55:00Z,2024-06-26T04:00:00Z,NIGHT,"POLYGON ((-34.49823 28.62535, -11.24442 25.426..."
3,MYD021KM.A2024178.0400.061.2024178213726.hdf,2024-06-26T04:00:00Z,2024-06-26T04:05:00Z,NIGHT,"POLYGON ((-37.46187 10.78717, -16.60438 7.8302..."
4,MYD021KM.A2024178.0405.061.2024178213412.hdf,2024-06-26T04:05:00Z,2024-06-26T04:10:00Z,NIGHT,"POLYGON ((-41.3058 -7.03763, -20.58997 -10.110..."
...,...,...,...,...,...
603,MYD021KM.A2024193.1840.061.2024194151104.hdf,2024-07-11T18:40:00Z,2024-07-11T18:45:00Z,DAY,"POLYGON ((-61.54154 62.98591, -102.1163 56.623..."
604,MYD021KM.A2024193.2005.061.2024194152830.hdf,2024-07-11T20:05:00Z,2024-07-11T20:10:00Z,DAY,"POLYGON ((-80.66352 13.39901, -101.55526 10.32..."
605,MYD021KM.A2024193.2010.061.2024194152502.hdf,2024-07-11T20:10:00Z,2024-07-11T20:15:00Z,DAY,"POLYGON ((-83.62082 31.49237, -107.10101 27.89..."
606,MYD021KM.A2024193.2015.061.2024194152417.hdf,2024-07-11T20:15:00Z,2024-07-11T20:20:00Z,DAY,"POLYGON ((-85.67897 49.58048, -115.71778 44.89..."
